In [12]:
# Collecting all the malayalam chars
malayalam_chars = [chr(code) for code in range(0x0D00, 0x0D7F + 1)]
malayalam_chars[:10], malayalam_chars[-10:], len(malayalam_chars)

(['ഀ', 'ഁ', 'ം', 'ഃ', 'ഄ', 'അ', 'ആ', 'ഇ', 'ഈ', 'ഉ'],
 ['൶', '൷', '൸', '൹', 'ൺ', 'ൻ', 'ർ', 'ൽ', 'ൾ', 'ൿ'],
 128)

In [ ]:
# Creating a new mapping starting from 256 to encode all malayalam chars
malayalam_chars_vocab = {256+i: list(char.encode("utf-8"))  for i, char in enumerate(malayalam_chars)}

In [20]:
def get_stats(ids: list[int]) -> dict[tuple[int, int], int]:
    """Takes a list of ints and find the occurance of each pair"""
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

def get_top_pair(stats: dict) -> tuple[int, int]:
    """Find the most occuring pair"""
    return max(stats, key=stats.get)

In [ ]:
def update_ids(ids: list[int], malayalam_chars_vocab) -> list[int]:
    """Merge the ids to new ids corresponding to malayalam_chars_vocab"""
    merged_tokens = []
    i = 0
    while i < len(tokens):
        if i < len(tokens)-1 and (tokens[i], tokens[i+1]) == pair_to_merge:
            merged_tokens.append(new_idx)
            i += 2
        else:
            merged_tokens.append(tokens[i])
            i += 1
    return merged_tokens
